* 파이썬의 메모리 관리는 가비지 콜렉터에 의해서 관리된다는 믿음이 있는데, 이것도 (어느 정도 예전에는 사실이었으나) 진짜는 아니다. 
* 파이썬은 Objective-C와 비슷하게 참조수(Reference Count)기반의 자동 메모리 관리 모델을 따르고 있다. 파이썬의 모든 변수는 값을 담는 영역이 아니라 객체에 바인딩 되는 이름이다. 
* 객체와 이름이 바인딩되면, 해당 객체는 그 이름에 의해 참조되는 것이고 이는 그 참조수를 1만큼 증가시키는 작용을 한다.
* 그 이름이 더 이상 해당 객체를 가리키지 않게 되는 경우, (변수 스코프를 벗어나거나 다른 객체에 바인딩 되는 경우) 참조수는 1이 감소하고, 그 결과로 참조수가 0이되면 해당 객체는 가비지 콜렉터의 도움없이 그 자리에서 즉시 파괴된다.  
* 파이썬의 이러한 메모리 관리방식으로부터 발생할 수 있는 함정, 메모리 누수가 생길 수 있는 케이스와 어떻게 이를 회피할 수 있는지를 알아본다

In [ ]:
import sys

In [16]:

class Foo:
  def __init__(self):
    self.value = 1
    print(f'Object({id(self)}:{self.__class__}) is being Created.')

  def __del__(self):
    print(f'Object({id(self)}:{self.__class__}) is being destroyed.')

a = Foo()
print('...')
a = 1
print('end')



Object(139976578569232:<class '__main__.Foo'>) is being Created.
...
Object(139976578569232:<class '__main__.Foo'>) is being destroyed.
end


- 위 코드를 보면 __del__() 메소드를 정의하여 객체가 삭제될 때 메세지를 출력하도록 했다. 
- 그리고 실제로 인스턴스를 생성한 다음, a라는 이름에 바인딩했다. 
- 곧이어 a라는 이름을 다른 객체, int 타입 1이라는 객체에 바인딩하게 되면 처음에 Foo()로 생성된 객체는 자신을 참조하는 이름이 0개가 되고 곧 GC에 의해서 제거된다. 

In [50]:
a = None
b = None

In [57]:
a = None
b = None

class Foo:
  def __init__(self):
    self.value = 1
    self.friend = None
    print(f'Object({hex(id(self))}:{self.__class__}) is being Created.')

  def __del__(self):
    del self.friend
    print(f'Object({hex(id(self))}:{self.__class__}) is being destroyed.')


def func(a):
  b = Foo()
  print(a)
  print(b)
  a.friend = b
  #b.friend = a

a = Foo()
func(a)
del a

Object(0x7f4ed22fdb10:<class '__main__.Foo'>) is being destroyed.
Object(0x7f4ed22ac090:<class '__main__.Foo'>) is being destroyed.
Object(0x7f4ed2253990:<class '__main__.Foo'>) is being Created.
Object(0x7f4ed2253c50:<class '__main__.Foo'>) is being Created.
Object(0x7f4ed2253c50:<class '__main__.Foo'>) is being destroyed.
Object(0x7f4ed2253990:<class '__main__.Foo'>) is being destroyed.


- 두 객체가 각각 자신의 속성으로 서로를 참조하고 있다. 따라서 a = None이 호출되는 시점에 원래 a 였던 객체의 참조수는 2에서 1로 줄어든다. 
- 하지만 남은 참조를 가지고 있던 이름 b는  None을 가리키고 있고, b.friend 라는 속성 이름 자체에 대한 접근이 막혔다. 
- 따라서 a.__del__() 이 호출될 수 없기 때문에 두 객체는 메모리에 계속 남아 메모리 누수가 발생하게 된다.

## 약한 참조
- 클래스 인스턴스의 속성이 만약 다른 클래스의 인스턴스를 참조하거나, 동일 클래스의 다른 인스턴스를 참조할 가능성이 높다면 이는 메모리 누수가 발생할 가능성이 매우 높은 지점이 된다. 
- 물론 파이썬 프로그램의 생애주기는 대부분 짧기 때문에 문제가 되지 않을 수 있지만, 서버와 같이 생애 주기가 길거나, 매우 많은 인스턴스를 생성하고 연결하는 동작을 하는 경우에 메모리 누수가 큰 문제가 될 수 있다.

## 약한 참조란
- 파이썬에서는 이러한 문제를 조금 간단히 처리하기 위해서 약한 참조를 제공한다.
- 약한 참조는 말 그대로 대상 객체를 참조는 하지만, 대상 객체에 대한 소유권을 주장하지 않는, 즉 reference count를 올리지 않는 참조를 말한다.
- 약한참조는 weakref 모듈의 ref 라는 클래스를 통해서 생성할 수 있다. 

In [58]:
class Foo:
  def __init__(self):
    self.value = 1
    self.friend = None
  def __del__(self):
    ## 여기서 딱히 friend 속성을 정리하지 않는다.
    print(f'Object({id(self):x}) is being destroyed.')

import weakref

a, b = Foo(), Foo()
a.friend = weakref.ref(b)
b.friend = weakref.ref(a)
b = None

a.friend()
a = None

Object(7f4ed2253a10) is being destroyed.
Object(7f4ed22534d0) is being destroyed.
